In [52]:
from collections import defaultdict
from tqdm import tqdm

import pickle
import pandas as pd

import requests

In [28]:
dis = pickle.load(open('dis.pkl', 'rb'))

In [29]:
print(len(dis))
dis[::2000]

34451


['3 Doors Down/1997 - 3 Doors Down (Demo Album)/01. Kryptonite.mp3',
 'Anthrax/1993 - Sound Of White Noise (320)/07 - 1000 Points Of Hate.mp3',
 'Bruce Springsteen/Live In Dublin/1-09 My Oklahoma Home.mp3',
 'Buckethead/2014 - Pike 084 - Whirlpool/08 - OO.mp3',
 'Cradle Of Filth/Thornography/04 Libertina Grimm.mp3',
 'Epica/2007 - The Divine Conspiracy/01 - Indigo (Prologue).mp3',
 'Frank Sinatra/1994 - The Song Is You/Cover/16.jpg',
 'Gary Moore/1989 - After the War (remaster 2003)/13. Over the Hills and Far Away (Live).mp3',
 'Hinder/2008 - Take It To The Limit/Hinder - 02 - Loaded And Alone.mp3',
 'Kamelot/2005 - The Black Halo (Limited Edition)/06. Abandoned.mp3',
 "Linkin Park/Road To Revolution Live At Milton Keynes/What I've Done.mp3",
 "Mogwai/2001 - Rock Action (Japanese Edition)/05 - You Don't Know Jesus.mp3",
 'Paradise Lost/Paradise Lost - (2009) Faith Divides Us - Death Unites Us (Special Editon)/09  Universal Dream.mp3',
 'Queen/(1976) A Day at the Races/10 Queen - Teo To

In [30]:
EXTS = {'mp3', 'flac', 'm3u', 'm4a', 'wma'}

In [31]:
exts = sorted({n.split('.')[-1].lower() for n in dis})
', '.join(exts)

'accurip, avi, bak, bmp, cue, db, flac, flv, gif, html, ini, jpeg, jpg, jpg_170x128, jpg_320x320, jpg_56x42, log, m3u, m4a, mkv, mmp3, mp3, mp4, mpg, nfo, pdf, pkl, png, rar, rtf, sfk, sfv, tif, txt, wma, wpl, zip'

In [32]:
music = list(filter(lambda n: n.split('.')[-1].lower() in EXTS, dis))

In [33]:
len(music)

27803

In [34]:
{len(m.split('/')) for m in music}

{3, 4}

In [35]:
all_music = defaultdict(dict)

for p in [m for m in music if len(m.split('/')) in (3, 4)]:
    band, album, *_, song = p.split('/')
    
    all_music[band].setdefault(album, [])
    all_music[band][album].append(song)

In [36]:
len(all_music)

134

In [37]:
all_music.keys()

dict_keys(['3 Doors Down', '30 Seconds To Mars', 'AC DC', 'Adele', 'Aerosmith', 'Alan Jackson', 'Alesana', 'Alter Bridge', 'Andy McKee', 'Angel Vivaldi', 'Animals as Leaders', 'Anthrax', 'Apocalyptica', 'Arch Enemy', 'Avenged Sevenfold', 'Bach - Complete Violin Sonatas - Arthur Grumiaux, Christiane Jaccottet, Philippe Mermoud', 'Black Sabbath', 'Bob Marley', 'Breaking Benjamin', 'Bruce Springsteen', 'Bryan Adams', 'Buckethead', 'Bullet For My Valentine', 'Children Of Bodom', 'Coldplay', 'Cradle Of Filth', 'Creed', 'Crossfade', 'Daughtry', 'Dead by April', 'Def Leppard', 'Dimmu Borgir', 'Disturbed', 'Dragonforce', 'Dream Theater', 'Eluveitie', 'Epica', 'Eric Clapton', 'Eternal Tears Of Sorrow', 'Eva Cassidy', 'Evanescence', 'Forever Slave', 'Frank Sinatra', 'Gary Moore', 'Green Day', 'Grey Heaven Fall', "Guns N' Roses", 'HIM', 'Hinder', 'Hollywood Undead', 'Insomnium', 'Iron Maiden', 'Jimi Hendrix', 'Joe Satriani', 'John Mayer', 'Kamelot', 'Kansas', 'Lacuna Coil', 'Lamb of God', 'Led Ze

In [38]:
all_music['3 Doors Down']

{'1997 - 3 Doors Down (Demo Album)': ['01. Kryptonite.mp3',
  '02. Loser.mp3',
  '03. Life On My Own.mp3',
  '04. Smack.mp3',
  '05. Down Poison.mp3',
  '06. Dead Love.mp3',
  '07. Wasted Me.mp3',
  '08. Man In My Mind.mp3',
  '09. By My Side.mp3',
  "10. Sarah Yellin' 86.mp3"],
 '2000 - The Better Life (Best Buy Bonus Tracks Version)': ['01. Kryptonite.mp3',
  '02. Loser.mp3',
  '03. Duck And Run.mp3',
  '04. Not Enough.mp3',
  '05. Be Like That.mp3',
  '06. Life Of My Own.mp3',
  '07. Better Life.mp3',
  '08. Down Poison.mp3',
  '09. By My Side.mp3',
  '10. Smack.mp3',
  '11. So I Need You.mp3',
  '12. Loser (Live).mp3',
  '13. Life Of My Own (Live).mp3',
  '14. Kryptonite (Live).mp3'],
 '2002 - Away From The Sun (Special Edition)': ["01. When I'm Gone.mp3",
  '02. Away From The Sun.mp3',
  "03. The Road I'm On.mp3",
  '04. Ticket To Heaven.mp3',
  '05. Running Out Of Days.mp3',
  '06. Here Without You.mp3',
  '07. I Feel You.mp3',
  '08. Dangerous Game.mp3',
  '09. Changes.mp3',
  '

In [65]:
API_BASE = 'http://127.0.0.1:8000'
ADD_BAND_URL = f'{API_BASE}/music/bands'
ADD_ALBUM_URL = f'{API_BASE}/music/albums'

In [66]:
for band, albums in tqdm(all_music.items()):
    res = requests.post(ADD_BAND_URL, data={'band': band}).json()
    band_id = res['band_id']
    for album, songs in albums.items():
        res = requests.post(ADD_ALBUM_URL, json={
            'album': album, 
            'band_id': band_id,
            'songs': songs,
        }).json()

100%|██████████| 134/134 [04:33<00:00,  2.04s/it]
